### Creating Dataloaders

In [3]:
import tensorflow as tf
import keras_tuner
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam,SGD

train_ds = keras.utils.image_dataset_from_directory(
    directory=r'D:\Gaussian_blur_bicubic_interpolation\Classification\HR\train',
    labels='inferred',
    label_mode='binary',
    batch_size=32,
    color_mode="rgb",
    seed=123,
    image_size=(240, 240))

validation_ds = keras.utils.image_dataset_from_directory(
    directory=r'D:\Gaussian_blur_bicubic_interpolation\Classification\HR\val',
    labels='inferred',
    label_mode='binary',
    batch_size=32,
    color_mode="rgb",
    seed=123,
    image_size=(240, 240))

Found 1550 files belonging to 2 classes.
Found 194 files belonging to 2 classes.


### Hyper parameter tuning

In [4]:
from tensorflow.keras.optimizers import Adam,SGD

def build_model(hp):
    resnet_model = Sequential()

    pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                       input_shape=(240,240,3),
                       pooling='avg',classes=2,
                       weights='imagenet')
    for layer in pretrained_model.layers:
            layer.trainable=False

    resnet_model.add(pretrained_model)
    resnet_model.add(Flatten())
    resnet_model.add(Dense(1, activation='sigmoid'))
    optimizer=hp.Choice('optimizer', values=['sgd', 'adam'])                
    # Conditional for each optimizer
    if optimizer == 'adam':
        learning_rate = hp.Choice("learning_rate", values=[0.001, 0.01, 0.1])
        resnet_model.compile(optimizer=Adam(lr=learning_rate),loss='binary_crossentropy',metrics=['accuracy'])
    elif optimizer == 'sgd':
        learning_rate = hp.Choice("learning_rate", values=[0.001, 0.01, 0.1])
        resnet_model.compile(optimizer=SGD(learning_rate=learning_rate),loss='binary_crossentropy',metrics=['accuracy'])
    return resnet_model

build_model(keras_tuner.HyperParameters())

In [5]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=1,
    max_retries_per_trial=1,
    max_consecutive_failed_trials=3,
    overwrite=True,
    directory=r"D:\\",
    project_name="hyper_parameter_results_thesis")

In [6]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
optimizer (Choice)
{'default': 'sgd', 'conditions': [], 'values': ['sgd', 'adam'], 'ordered': False}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.01, 0.1], 'ordered': True}


In [7]:
tuner.search(train_ds, epochs=5, validation_data=validation_ds)

Trial 6 Complete [00h 10m 26s]
val_accuracy: 0.8917526006698608

Best val_accuracy So Far: 0.8917526006698608
Total elapsed time: 01h 02m 07s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


### Results  of Hyper parameter tuning

In [8]:
tuner.results_summary()

Results summary
Results in D:\\hyper_parameter_results_thesis
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 05 summary
Hyperparameters:
optimizer: adam
learning_rate: 0.001
Score: 0.8917526006698608

Trial 01 summary
Hyperparameters:
optimizer: adam
learning_rate: 0.1
Score: 0.876288652420044

Trial 03 summary
Hyperparameters:
optimizer: adam
learning_rate: 0.01
Score: 0.8659793734550476

Trial 02 summary
Hyperparameters:
optimizer: sgd
learning_rate: 0.01
Score: 0.8350515365600586

Trial 00 summary
Hyperparameters:
optimizer: sgd
learning_rate: 0.1
Score: 0.8247422575950623

Trial 04 summary
Hyperparameters:
optimizer: sgd
learning_rate: 0.001
Score: 0.7628865838050842
